# google colab 세팅
- Runtime (구글 코랩, 데이터셋 칼럼 2000개 기준)
    - CPU, 고용량 RAM > 너무 느림 (40m)
    - TPU로 변경 (25m)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Hanium_private/model/

/content/drive/MyDrive/Hanium_private/model


# DataLoader

In [ ]:
import json
import os
import pandas as pd

In [ ]:
base_dir = ['2020-02-019_document_summary_Sample/']

for basedir in base_dir:
  filelist = sorted(os.listdir(basedir))

- Json - Data Loader: 22min 소요

In [ ]:
# 여러 JSON 파일들을 DataFrame에 로드하는 함수
def load_data_from_jsons(basedir):
    for directory in basedir:
      data = []
      filelist = sorted(os.listdir(directory))
      for filename in filelist:
          if filename.endswith(".json"):
              with open(os.path.join(directory, filename), 'r', encoding="utf-8") as file:
                  entry = json.load(file)
                  data.append([
                      entry['passage_id'],
                      entry['metadata']['doc_id'],
                      entry['metadata']['doc_name'],
                      entry['passage'],
                      entry['metadata']['author'],
                      entry['metadata']['kdc_label'],
                      entry['summary']
                  ])
      return pd.DataFrame(data, columns=['id', 'title_id', 'title', 'passage', 'author', 'type', 'summary'])

# 데이터셋 로드
for filename in filelist:
  book_df = load_data_from_jsons(base_dir)

In [ ]:
book_df.head()

,id,title_id,title,passage,author,type,summary
0,CNTS-00047965957_40_41-0,CNTS-00047965957,후진타오 시대의 중국정치,선거개혁도 유사한 문제가 있다. 유권자들은 1979년 현급 지방인대 직접선거 개혁...,조영남,정치학,1979년 지방인대 직접선거 개혁이후 유권자들은 적극적이고 능동적으로 선거에 참여...
1,CNTS-00047966059_168,CNTS-00047966059,열림과 닫힘,우리는 종교가 인간의 꿈을 실현하는 ‘해답의 현실화’라는 사실도 언급합니다. 우리는...,정진홍 지음,종교,종교는 해답의 현실화라고도 언급되어 우리는 온전함과 평안함을 누리기도 한다. 하지만...
2,CNTS-00047966059_371,CNTS-00047966059,열림과 닫힘,그런데 절대적인 힘과의 만남은 그 힘이 제시하는 해답에 대한 무조건적인 추종을 수용...,정진홍 지음,종교,‘신과의 만남’이란 ‘다가가고 싶지만 멀리하고 싶은’ 상태에서 이루어지는 독특한 ‘...
3,CNTS-00047966059_451,CNTS-00047966059,열림과 닫힘,‘같은데 다르다’라든지 ‘다르지만 같다’라고 하는 것을 충분히 서술할 수 있어야 비...,정진홍 지음,종교,종교문화가 여타 문화 일반과 ‘다른 것’이기보다 삶의 한 측면을 드러내는 것이라고 ...
4,CNTS-00047966059_497,CNTS-00047966059,열림과 닫힘,그런데 주목할 것은 종교문화의 죽음담론은 결코 죽음을 종국으로 설정하지 않는다는 사...,정진홍 지음,종교,종교문화는 죽음을 죽음 이후를 다루는 담론으로 바라봄으로써 죽음물음이 현실성을 지니...


In [ ]:
import sqlite3

def sqlite_append(df):
  con = sqlite3.connect("data/books.db")

  df.to_sql('sample', con, if_exists='append', index=False)
  con.commit()
  con.close()

sqlite_append(book_df)

## 결과 확인 
- db 확인

In [ ]:
conn = sqlite3.connect('data/books.db')
cur = conn.cursor()

cur.execute("SELECT COUNT(*) FROM sample;")
print(cur.fetchall())

[(2000,)]


In [ ]:
conn.close()